In [1]:
# default_exp modules.data

In [2]:
#export
import numpy as np
import glob
import os
import uproot as ur
import time
from multiprocessing import Process, Queue, set_start_method
import compress_pickle as pickle
from scipy.stats import circmean
import random
import itertools

import pandas as pd

In [3]:
pd.set_option('display.max_columns', None)

# GraphDataGenerator

> API details.

In [ ]:
data_dir = '/clusterfs/ml4hep/mpettee/ml4pions/data/'
out_dir = '/clusterfs/ml4hep/mpettee/ml4pions/data/tracks_withcuts/'
pi0_files = np.sort(glob.glob(data_dir+'*pi0*/*.root'))[10:12]
pion_files = np.sort(glob.glob(data_dir+'*pion*/*.root'))[10:12]

In [ ]:
cell_geo_df = ur.open(data_dir+"cell_geo.root", library="pd")["CellGeo"].arrays(library="pd")
event_data = ur.open(pion_files[0])["EventTree"].arrays(library="np")
# test_pi0_df = pd.DataFrame(test_pi0)

In [4]:
#hide
from nbdev.showdoc import *

In [3]:
#export
class GraphDataGenerator:
    """
    DataGenerator class for extracting and formating data from list of root files
    This data generator uses the cell_geo file to create the input graph structure
    """
    def __init__(self,
                 pi0_file_list: list,
                 pion_file_list: list,
                 cellGeo_file: str,
                 batch_size: int,
                 shuffle: bool = True,
                 num_procs = 32,
                 preprocess = False,
                 output_dir = None):
        """Initialization"""

        self.preprocess = preprocess
        self.output_dir = output_dir

        if self.preprocess and self.output_dir is not None:
            self.pi0_file_list = pi0_file_list
            self.pion_file_list = pion_file_list
            assert len(pi0_file_list) == len(pion_file_list)
            self.num_files = len(self.pi0_file_list)
        else:
            self.file_list = pi0_file_list
            self.num_files = len(self.file_list)
        
        self.cellGeo_file = cellGeo_file
        
        self.cellGeo_data = ur.open(self.cellGeo_file)['CellGeo']
        self.geoFeatureNames = self.cellGeo_data.keys()[1:9]
        self.nodeFeatureNames = ['cluster_cell_E', *self.geoFeatureNames[:-2]]
        self.edgeFeatureNames = self.cellGeo_data.keys()[9:]
        self.num_nodeFeatures = len(self.nodeFeatureNames)
        self.num_edgeFeatures = len(self.edgeFeatureNames)
        self.cellGeo_data = self.cellGeo_data.arrays(library='np')
        self.cellGeo_ID = self.cellGeo_data['cell_geo_ID'][0]
        self.sorter = np.argsort(self.cellGeo_ID)
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        if self.shuffle: np.random.shuffle(self.file_list)
        
        self.num_procs = np.min([num_procs, self.num_files])
        self.procs = []

        if self.preprocess and self.output_dir is not None:
            os.makedirs(self.output_dir, exist_ok=True)
            self.preprocess_data()
            
#         print(self.nodeFeatureNames)
#         print(self.edgeFeatureNames)

    def get_cluster_calib(self, event_data, event_ind, cluster_ind):
        """ Reading cluster calibration energy """ 
            
        cluster_calib_E = event_data['cluster_ENG_CALIB_TOT'][event_ind][cluster_ind]

        if cluster_calib_E <= 0:
            return None

        return np.log10(cluster_calib_E)
            
    def get_nodes(self, event_data, event_ind, cluster_ind):
        """ Reading Node features """ 

        cell_IDs = event_data['cluster_cell_ID'][event_ind][cluster_ind]
        cell_IDmap = self.sorter[np.searchsorted(self.cellGeo_ID, cell_IDs, sorter=self.sorter)]
        
        nodes = np.log10(event_data['cluster_cell_E'][event_ind][cluster_ind])
        global_node = np.log10(event_data['cluster_E'][event_ind][cluster_ind])
        
        # Scaling the cell_geo_sampling by 28
        nodes = np.append(nodes, self.cellGeo_data['cell_geo_sampling'][0][cell_IDmap]/28.)
        for f in self.nodeFeatureNames[2:4]:
            nodes = np.append(nodes, self.cellGeo_data[f][0][cell_IDmap])
        # Scaling the cell_geo_rPerp by 3000
        nodes = np.append(nodes, self.cellGeo_data['cell_geo_rPerp'][0][cell_IDmap]/3000.)
        for f in self.nodeFeatureNames[5:]:
            nodes = np.append(nodes, self.cellGeo_data[f][0][cell_IDmap])

        nodes = np.reshape(nodes, (len(self.nodeFeatureNames), -1)).T
        cluster_num_nodes = len(nodes)
    
        # add dummy placeholder nodes for track features (not used in cluster cell nodes)
        nodes = np.hstack((nodes, np.zeros((cluster_num_nodes, 4))))
        
#         print("nodes", nodes)
#         print("global_node", np.array([global_node]))
#         print("cluster_num_nodes", cluster_num_nodes)
#         print(cell_IDmap)
#         raise Exception("asdfasdfasdf")
        return nodes, np.array([global_node]), cluster_num_nodes, cell_IDmap
    
    
    
    
    # WIP ----------------------------------------------------------------
    
    
    
    def get_track_node(self, event_data, event_index, track_index):
        """
        Creates node features for tracks
        Inputs:
        
        Returns:
            1 Dimensional array of node features for a single node
                NOTE the cluster get_node function is a 2D array of multiple nodes
                This function is used in a for loop so the end result is a 2D array
        """
        node_features = np.array(event_data["trackPt"][event_index][track_index])
        node_features = np.append(node_features, event_data["trackZ0"][event_index][track_index])
        node_features = np.append(node_features, event_data["trackEta_EMB2"][event_index][track_index])
        node_features = np.append(node_features, event_data["trackPhi_EMB2"][event_index][track_index])
        node_features = np.reshape(node_features, (len(node_features))).T
        
#         print("node_features before", node_features)
        # add dummy placeholder nodes for track features (not used in track cell nodes)
        node_features = np.hstack((np.zeros(7), node_features))
        
#         print(node_features)
#         raise Exception("asdfasdfasdf")
        return node_features
    
    def get_track_edges(self, num_track_nodes, start_index):
        """
        Creates the edge senders and recievers and edge features
        Inputs:
        (int) num_track_nodes: number of track nodes
        (int) start_index: the index of senders/recievers to start with. We should start with num_cluster_edges+1 to avoid overlap

        Returns:
        (np.array) edge_features:
        (np.array) senders:
        (np.array) recievers:
        """
        # Full Connected tracks
        # since we are fully connected, the order of senders and recievers doesn't matter
        # we just need to count each node - edges will have a placeholder feature
        connections = list(itertools.permutations(range(start_index, start_index + num_track_nodes),2))
        for i in range(5):
            connections.append((i, i))

        senders = np.array([x[0] for x in connections])
        recievers = np.array([x[0] for x in connections])
        edge_features = np.zeros((len(connections), 10))

        return senders, recievers, edge_features
    
    
    
    
    # end WIP ----------------------------------------------------------------
    
    
    
    
    def get_edges(self, cluster_num_nodes, cell_IDmap):
        """ 
        Reading edge features 
        Returns senders, receivers, and edges    
        """ 
        
        edge_inds = np.zeros((cluster_num_nodes, self.num_edgeFeatures))
        for i, f in enumerate(self.edgeFeatureNames):
            edge_inds[:, i] = self.cellGeo_data[f][0][cell_IDmap]
        edge_inds[np.logical_not(np.isin(edge_inds, cell_IDmap))] = np.nan
        
        senders, edge_on_inds = np.isin(edge_inds, cell_IDmap).nonzero()
        cluster_num_edges = len(senders)
        edges = np.zeros((cluster_num_edges, self.num_edgeFeatures))
        edges[np.arange(cluster_num_edges), edge_on_inds] = 1
        
        cell_IDmap_sorter = np.argsort(cell_IDmap)
        rank = np.searchsorted(cell_IDmap, edge_inds , sorter=cell_IDmap_sorter)
        receivers = cell_IDmap_sorter[rank[rank!=cluster_num_nodes]]
        
        return senders, receivers, edges

    def preprocessor(self, worker_id):
        """
        Prerocessing root file data for faster data 
        generation during multiple training epochs
        """
        file_num = worker_id
        while file_num < self.num_files:
            print(f"Processing file number {file_num}")
            file = self.pion_file_list[file_num]
            event_data = np.load(file, allow_pickle=True).item()
            num_events = len(event_data[[key for key in event_data.keys()][0]])

            preprocessed_data = []

            for event_ind in range(num_events):
                num_clusters = event_data['nCluster'][event_ind]
                
                for i in range(num_clusters):
                    cluster_calib_E = self.get_cluster_calib(event_data, event_ind, i)
                    
                    if cluster_calib_E is None:
                        continue
                        
                    nodes, global_node, cluster_num_nodes, cell_IDmap = self.get_nodes(event_data, event_ind, i)
                    senders, receivers, edges = self.get_edges(cluster_num_nodes, cell_IDmap)
                    
                    
                    
                    
                    
                    
                    # WIP add track nodes and edges ----------------------------------------------------------------
                    track_nodes = np.empty((0, 11))
                    num_tracks = event_data['nTrack'][event_ind]
                    for track_index in range(num_tracks):
#                         print("get func reshaped", self.get_track_node(event_data, event_ind, track_index).reshape(1, -1))
#                         print("track_nodes", track_nodes)
#                         raise Exception("stop")
                        np.append(track_nodes, self.get_track_node(event_data, event_ind, track_index).reshape(1, -1), axis=0)
                    
                    track_senders, track_receivers, track_edge_features = self.get_track_edges(len(track_nodes), cluster_num_nodes)
                    
                    
                    
#                     print("old nodes", nodes)
#                     print("track nodes", track_nodes)
                    
                    
                    # append on the track nodes and edges to the cluster ones
                
                    nodes = np.append(nodes, np.array(track_nodes), axis=0)
                    edges = np.append(edges, track_edge_features, axis=0)
                    senders = np.append(senders, track_senders, axis=0)
                    receivers = np.append(receivers, track_receivers, axis=0)
                    
                    # end WIP ----------------------------------------------------------------
                
                    graph = {'nodes': nodes.astype(np.float32), 'globals': global_node.astype(np.float32),
                        'senders': senders.astype(np.int32), 'receivers': receivers.astype(np.int32),
                        'edges': edges.astype(np.float32)}
                    target = np.reshape([cluster_calib_E.astype(np.float32), 1], [1,2])

                    preprocessed_data.append((graph, target))

            file = self.pi0_file_list[file_num]
            event_data = np.load(file, allow_pickle=True).item()
            num_events = len(event_data[[key for key in event_data.keys()][0]])

            for event_ind in range(num_events):
                num_clusters = event_data['nCluster'][event_ind]
                
                for i in range(num_clusters):
                    cluster_calib_E = self.get_cluster_calib(event_data, event_ind, i)
                    
                    if cluster_calib_E is None:
                        continue
                        
                    nodes, global_node, cluster_num_nodes, cell_IDmap = self.get_nodes(event_data, event_ind, i)
                    senders, receivers, edges = self.get_edges(cluster_num_nodes, cell_IDmap)         
                    
                    # WIP add track nodes and edges ----------------------------------------------------------------
                    track_nodes = np.empty((0, 11))
                    num_tracks = event_data['nTrack'][event_ind]
                    for track_index in range(num_tracks):
                        np.append(track_nodes, self.get_track_node(event_data, event_ind, track_index).reshape(1, -1), axis=0)
                    
                    track_senders, track_receivers, track_edge_features = self.get_track_edges(len(track_nodes), cluster_num_nodes)
                    
                    
                    # append on the track nodes and edges to the cluster ones
#                     if track_nodes:
                    nodes = np.append(nodes, np.array(track_nodes), axis=0)
                    edges = np.append(edges, track_edge_features, axis=0)
                    senders = np.append(senders, track_senders, axis=0)
                    receivers = np.append(receivers, track_receivers, axis=0)
                    
                    
#                     print("nodes", nodes)
#                     print("edges", edges)
#                     raise Exception("asdf")
                    
                    # end WIP ----------------------------------------------------------------
                    
                    graph = {'nodes': nodes.astype(np.float32), 'globals': global_node.astype(np.float32),
                        'senders': senders.astype(np.int32), 'receivers': receivers.astype(np.int32),
                        'edges': edges.astype(np.float32)}
                    target = np.reshape([cluster_calib_E.astype(np.float32), 0], [1,2])

                    preprocessed_data.append((graph, target))

            random.shuffle(preprocessed_data)

            pickle.dump(preprocessed_data, open(self.output_dir + f'data_{file_num:03d}.p', 'wb'), compression='gzip')
            
            print(f"Finished processing {file_num} files")
            file_num += self.num_procs

    def preprocess_data(self):
        print('\nPreprocessing and saving data to {}'.format(self.output_dir))
        for i in range(self.num_procs):
            p = Process(target=self.preprocessor, args=(i,), daemon=True)
            p.start()
            self.procs.append(p)
        
        for p in self.procs:
            p.join()

        self.file_list = [self.output_dir + f'data_{i:03d}.p' for i in range(self.num_files)]

    def preprocessed_worker(self, worker_id, batch_queue):
        batch_graphs = []
        batch_targets = []

        file_num = worker_id
        while file_num < self.num_files:
            file_data = pickle.load(open(self.file_list[file_num], 'rb'), compression='gzip')

            for i in range(len(file_data)):
                batch_graphs.append(file_data[i][0])
                batch_targets.append(file_data[i][1])
                    
                if len(batch_graphs) == self.batch_size:
                    batch_targets = np.reshape(np.array(batch_targets), [-1,2]).astype(np.float32)
                    
                    batch_queue.put((batch_graphs, batch_targets))
                    
                    batch_graphs = []
                    batch_targets = []

            file_num += self.num_procs
                    
        if len(batch_graphs) > 0:
            batch_targets = np.reshape(np.array(batch_targets), [-1,2]).astype(np.float32)
            
            batch_queue.put((batch_graphs, batch_targets))

    def worker(self, worker_id, batch_queue):
        if self.preprocess:
            self.preprocessed_worker(worker_id, batch_queue)
        else:
            raise Exception('Preprocessing is required for combined classification/regression models.')
        
    def check_procs(self):
        for p in self.procs:
            if p.is_alive(): return True
        
        return False

    def kill_procs(self):
        for p in self.procs:
            p.kill()

        self.procs = []
    
    def generator(self):
        """
        Generator that returns processed batches during training
        """
        batch_queue = Queue(2 * self.num_procs)
            
        for i in range(self.num_procs):
            p = Process(target=self.worker, args=(i, batch_queue), daemon=True)
            p.start()
            self.procs.append(p)
        
        while self.check_procs() or not batch_queue.empty():
            try:
                batch = batch_queue.get(True, 0.0001)
            except:
                continue
            
            yield batch
        
        for p in self.procs:
            p.join()

In [6]:
data_dir = '/clusterfs/ml4hep/mpettee/ml4pions/data/'
out_dir = '/clusterfs/ml4hep/mpettee/ml4pions/data/tracks_withcuts/'
pi0_files = np.sort(glob.glob(data_dir+'*pi0*/*.npy'))#[10:11]
pion_files = np.sort(glob.glob(data_dir+'*pion*/*.npy'))#[10:11]

In [ ]:
# data_dir = '/clusterfs/ml4hep/mpettee/ml4pions/data/'
# out_dir = '/clusterfs/ml4hep/mpettee/ml4pions/data/tracks_withcuts/'
# pi0_files = np.sort(glob.glob(data_dir+'*pi0*/*.root'))[:2]
# pion_files = np.sort(glob.glob(data_dir+'*pion*/*.root'))[:2]

In [ ]:
# np.load(pi0_files[0], allow_pickle=True).item().keys()

In [7]:
data_gen = GraphDataGenerator(pion_file_list=pion_files, 
                              pi0_file_list=pi0_files,
                              cellGeo_file=data_dir+'cell_geo.root',
                              batch_size=32,
                              shuffle=False,
                              num_procs=32,
                              preprocess=True,
                              output_dir=out_dir)

# gen = data_gen.generator()


Preprocessing and saving data to /clusterfs/ml4hep/mpettee/ml4pions/data/tracks_withcuts/
Processing file number 0
Processing file number 1
Processing file number 2
Processing file number 3
Processing file number 4
Processing file number 5
Processing file number 6
Processing file number 7
Processing file number 8
Processing file number 9


Process Process-3:
Process Process-1:
Process Process-2:
Process Process-10:
Process Process-7:
Process Process-8:
Process Process-9:
Process Process-4:
Process Process-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
TypeError: int() argument must be a string, a bytes-like object or a number, not 'STLVector'
Traceback (most recent call last):
Traceback (most recent call last):
  File "/clusterfs/ml4hep/mpettee/miniconda3/envs/nbdev/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/clusterfs/ml4hep/mpettee/miniconda3/envs/nbdev/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/clusterfs/ml4hep/mpettee/miniconda3/envs/nbdev/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/clusterfs/ml4hep/mpettee/minicon

KeyboardInterrupt: 

  File "/clusterfs/ml4hep/mpettee/miniconda3/envs/nbdev/lib/python3.9/site-packages/numpy/lib/arraysetops.py", line 261, in unique
    ret = _unique1d(ar, return_index, return_inverse, return_counts)
  File "/clusterfs/ml4hep/mpettee/miniconda3/envs/nbdev/lib/python3.9/site-packages/numpy/lib/arraysetops.py", line 319, in _unique1d
    perm = ar.argsort(kind='mergesort' if return_index else 'quicksort')
ValueError: setting an array element with a sequence.
KeyboardInterrupt
KeyboardInterrupt
  File "/clusterfs/ml4hep/mpettee/miniconda3/envs/nbdev/lib/python3.9/site-packages/numpy/lib/arraysetops.py", line 314, in _unique1d
    ar = np.asanyarray(ar).flatten()
KeyboardInterrupt
KeyboardInterrupt


In [ ]:
next(data_gen.generator())[0]

In [ ]:
a = np.append(np.empty((0, 11)), np.array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  5.69970337e+02,
        6.30014613e-02, -4.63907570e-02,  5.21522835e-02]).reshape(1, -1), axis=0)

In [ ]:
np.append(np.append(a, a, axis=0), a, axis=0)

In [ ]:
out_dir = '/clusterfs/ml4hep/mfong/ML4Pions/graph_data_tracks/'
pi0_files = np.sort(glob.glob(data_dir+'*pi0*/*.root'))[20:30]
pion_files = np.sort(glob.glob(data_dir+'*pion*/*.root'))[20:30]
data_gen_test = GraphDataGenerator(pion_file_list=pion_files, 
                                   pi0_file_list=pi0_files,
                                   cellGeo_file=data_dir+'CellGeo.neighbours.root',
                                   batch_size=32,
                                   shuffle=False,
                                   num_procs=32,
                                   preprocess=True,
                                   output_dir=out_dir)